Relevant directions from https://adventofcode.com/2023/day/2 part one -

You play several games and record the information from each game (your puzzle input). Each game is listed with its ID number (like the 11 in Game 11: ...) followed by a semicolon-separated list of subsets of cubes that were revealed from the bag (like 3 red, 5 green, 4 blue).

For example, the record of a few games might look like this:

```
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
```

In game 1, three sets of cubes are revealed from the bag (and then put back again). The first set is 3 blue cubes and 4 red cubes; the second set is 1 red cube, 2 green cubes, and 6 blue cubes; the third set is only 2 green cubes.

The Elf would first like to know which games would have been possible if the bag contained only 12 red cubes, 13 green cubes, and 14 blue cubes?

In the example above, games 1, 2, and 5 would have been possible if the bag had been loaded with that configuration. However, game 3 would have been impossible because at one point the Elf showed you 20 red cubes at once; similarly, game 4 would also have been impossible because the Elf showed you 15 blue cubes at once. If you add up the IDs of the games that would have been possible, you get **8**.

Determine which games would have been possible if the bag had been loaded with only 12 red cubes, 13 green cubes, and 14 blue cubes. What is the sum of the IDs of those games?

In [2]:
let text = r#"
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
"#;
let lines: Vec<&str> = text.trim().lines().collect();
lines

["Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green", "Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue", "Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red", "Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red", "Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green"]

In [3]:
#[derive(Debug)]
struct Bag {
    red: i32,
    green: i32,
    blue: i32,
}

impl Bag {
    fn new(red: i32, green: i32, blue: i32) -> Bag {
        Bag { red, green, blue }
    }

    fn from_string(s: &str) -> Result<Bag, &'static str> {
        let mut red = 0;
        let mut green = 0;
        let mut blue = 0;

        for part in s.split(',') {
            let parts: Vec<&str> = part.trim().split_whitespace().collect();
            if parts.len() != 2 {
                return Err("Invalid input format");
            }

            let count = parts[0].parse::<i32>().map_err(|_| "Invalid count")?;
            match parts[1] {
                "red" => red = count,
                "green" => green = count,
                "blue" => blue = count,
                _ => return Err("Unknown color"),
            }
        }

        Ok(Bag::new(red, green, blue))
    }

    fn can_contain(&self, other: &Bag) -> bool {
        self.red >= other.red && self.green >= other.green && self.blue >= other.blue
    }
}

let example = Bag::from_string("8 green, 6 blue, 20 red").unwrap();
example

Bag { red: 20, green: 8, blue: 6 }

In [4]:
let max_bag = Bag::new(12, 13, 14);
max_bag.can_contain(&example)

false

In [5]:
#[derive(Debug)]
struct Game {
    id: i32,
    bags: Vec<Bag>,
}

impl Game {
    fn from_string(s: &str) -> Result<Game, &'static str> {
        let parts: Vec<&str> = s.split(':').collect();
        if parts.len() != 2 {
            return Err("Invalid input format");
        }

        let id = parts[0].split_whitespace().last().ok_or("Invalid ID")?
            .parse::<i32>().map_err(|_| "ID is not a number")?;

        let bag_parts = parts[1].split(';');
        let mut bags = Vec::new();

        for part in bag_parts {
            match Bag::from_string(part) {
                Ok(bag) => bags.push(bag),
                Err(e) => return Err(e),
            }
        }

        Ok(Game { id, bags })
    }

    fn check(&self, max_bag: &Bag) -> bool {
        self.bags.iter().all(|bag| max_bag.can_contain(bag))
    }
}

let game = Game::from_string("Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green\n").unwrap();
game

Game { id: 5, bags: [Bag { red: 6, green: 3, blue: 1 }, Bag { red: 1, green: 2, blue: 2 }] }

In [6]:
game.check(&max_bag)

true

In [7]:
let mut games = Vec::new();
for line in &lines {
    let game = Game::from_string(&line).unwrap();
    games.push(game);
};
games

[Game { id: 1, bags: [Bag { red: 4, green: 0, blue: 3 }, Bag { red: 1, green: 2, blue: 6 }, Bag { red: 0, green: 2, blue: 0 }] }, Game { id: 2, bags: [Bag { red: 0, green: 2, blue: 1 }, Bag { red: 1, green: 3, blue: 4 }, Bag { red: 0, green: 1, blue: 1 }] }, Game { id: 3, bags: [Bag { red: 20, green: 8, blue: 6 }, Bag { red: 4, green: 13, blue: 5 }, Bag { red: 1, green: 5, blue: 0 }] }, Game { id: 4, bags: [Bag { red: 3, green: 1, blue: 6 }, Bag { red: 6, green: 3, blue: 0 }, Bag { red: 14, green: 3, blue: 15 }] }, Game { id: 5, bags: [Bag { red: 6, green: 3, blue: 1 }, Bag { red: 1, green: 2, blue: 2 }] }]

In [8]:
for game in &games {
    println!("ID {}. Check: {}", &game.id, game.check(&max_bag));
}

ID 1. Check: true
ID 2. Check: true
ID 3. Check: false
ID 4. Check: false
ID 5. Check: true


()

In [9]:
let mut id_sum = 0;
for game in &games {
    if game.check(&max_bag) {
        id_sum += &game.id
    }
}
id_sum

8